In [210]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import boxscoreplayertrackv2
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import playergamelogs
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.static import teams
from json import JSONDecodeError
import pandas as pd
import requests
import time

In [98]:
def str_to_mins(inp):
    lst = inp.split(":")
    sec = int(lst[0])*60 + int(lst[1])
    return sec

In [149]:
games = pd.read_csv("all_games_since_1997.csv")
boxscores = pd.read_csv("all_boxscores_concatenated.csv")
season_stats = pd.read_csv("player-seasons_per_possession.csv")

In [190]:
playernames = list(boxscores["PLAYER_ID"].unique())

In [222]:
len(playernames)

2376

In [218]:
lst = []
time_taken = [10]
weird_ids = []
start = timeit.default_timer()
verbose = True
for i in range(len(playernames)):
    time.sleep(0.36)
    player = playernames[i]
    successful = False
    decode_error = False
    while not successful:
        try:
            get_start = timeit.default_timer()
            pl = commonplayerinfo.CommonPlayerInfo(player, timeout=round(2*sum(time_taken)/len(time_taken), 0))
            get_end = timeit.default_timer()
            time_taken.append(get_end-get_start)
            successful = True
        except JSONDecodeError:
            weird_ids.append(player)
            decode_error = True
            break
        except:
            if verbose:
                print(f"Timeout error on iteration {i}, taking a 3 second break before resuming")
            time.sleep(3)
        
    if decode_error: continue
    players = pl.get_data_frames()[0]
    lst.append(players)
    if i%50 == 1:
        stop = timeit.default_timer()
        if verbose:
            print(f"{i-1}/{len(playernames)} done: {round(i/len(playernames)*100,1)}% ... has taken {round((stop-start)/60,0)} min, total projected: {round((stop-start)/(i/len(playernames))/60, 2)} min, remaining projected: {round(((stop-start)/(i/len(playernames))-(stop-start))/60, 2)} min")
boxscores_concat = pd.concat(lst, axis = 0, ignore_index = True)
boxscores_concat.to_csv(f"commonplayerinfo.csv",index=False)


0/2376 done: 0.0% ... has taken 0.0 min, total projected: 39.43 min, remaining projected: 39.42 min
50/2376 done: 2.1% ... has taken 0.0 min, total projected: 22.88 min, remaining projected: 22.39 min
100/2376 done: 4.3% ... has taken 1.0 min, total projected: 23.19 min, remaining projected: 22.2 min
Timeout error on iteration 147, taking a 3 second break before resuming
150/2376 done: 6.4% ... has taken 2.0 min, total projected: 24.21 min, remaining projected: 22.67 min
200/2376 done: 8.5% ... has taken 2.0 min, total projected: 24.12 min, remaining projected: 22.08 min
250/2376 done: 10.6% ... has taken 3.0 min, total projected: 24.29 min, remaining projected: 21.72 min
300/2376 done: 12.7% ... has taken 3.0 min, total projected: 24.22 min, remaining projected: 21.15 min
Timeout error on iteration 306, taking a 3 second break before resuming
400/2376 done: 16.9% ... has taken 4.0 min, total projected: 24.73 min, remaining projected: 20.56 min
Timeout error on iteration 440, taking a 

In [150]:
games.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'],
      dtype='object')

In [151]:
boxscores.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'SPD', 'DIST',
       'ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS', 'AST', 'CFGM',
       'CFGA', 'CFG_PCT', 'UFGM', 'UFGA', 'UFG_PCT', 'FG_PCT', 'DFGM', 'DFGA',
       'DFG_PCT'],
      dtype='object')

In [152]:
season_stats.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'AGE', 'GP',
       'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV',
       'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',
       'NBA_FANTASY_PTS', 'DD2', 'TD3', 'GP_RANK', 'W_RANK', 'L_RANK',
       'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK',
       'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK',
       'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK',
       'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK',
       'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK',
       'TD3_RANK', 'CFID', 'CFPARAMS', 'SEASON'],
      dtype='object')

In [145]:
list_all_game_ids = list(games["GAME_ID"].unique())

In [186]:
master_dict = {k:{"home":dict(), "away":dict()} for k in list_all_game_ids[2000:2002]}
for game_id in list_all_game_ids[2000:2002]:
    relevant_games = games[(games['GAME_ID'] == game_id)]
    relevant_boxscores = boxscores[(boxscores['GAME_ID'] == game_id)]
    relevant_games.reset_index(drop=True, inplace=True)
    current_season = int(str(relevant_games.loc[0, "SEASON_ID"])[1:])
    print(current_season)
    
    # second step.... add the appropriate team id to home vs away
    if "@" in relevant_games.loc[0, "MATCHUP"]:
        master_dict[game_id]["away"]["team_id"] = relevant_games.loc[0, "TEAM_ID"]
        away_team_id = relevant_games.loc[0, "TEAM_ID"]
        master_dict[game_id]["home"]["team_id"] = relevant_games.loc[1, "TEAM_ID"]
        home_team_id = relevant_games.loc[1, "TEAM_ID"]
    else:
        master_dict[game_id]["away"]["team_id"] = relevant_games.loc[1, "TEAM_ID"]
        away_team_id = relevant_games.loc[1, "TEAM_ID"]
        master_dict[game_id]["home"]["team_id"] = relevant_games.loc[0, "TEAM_ID"]
        home_team_id = relevant_games.loc[0, "TEAM_ID"]
    
    #start logging the players
    master_dict[game_id]["home"]["players"] = dict()
    master_dict[game_id]["away"]["players"] = dict()
    for row in relevant_boxscores.itertuples():
        sec_played = str_to_mins(row.MIN)
        if sec_played > 0:
            if row.TEAM_ID == home_team_id:
                master_dict[game_id]["home"]["players"][row.PLAYER_NAME] = {"sec_played":sec_played, "stats":dict()}
                print(row.PLAYER_NAME)
                display(season_stats.loc[(season_stats["SEASON"] == current_season-1) & (season_stats["PLAYER_ID"] == row.PLAYER_ID)])
            else:
                master_dict[game_id]["away"]["players"][row.PLAYER_NAME] = {"sec_played":sec_played, "stats":dict()}
        

1999
Grant Hill


,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,...,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,CFID,CFPARAMS,SEASON
620,255,Grant Hill,1.610613e+09,DET,26.0,50,29,21,0.58,0.537,...,377,130,14,76,15,38,3,5,"255,1610612765",1998


Christian Laettner


,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,...,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,CFID,CFPARAMS,SEASON
533,363,Christian Laettner,1.610613e+09,DET,29.0,16,13,3,0.813,0.524,...,264,130,187,137,116,174,12,5,"363,1610612765",1998


Terry Mills


...

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,...,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,CFID,CFPARAMS,SEASON


Flip Saunders


,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,...,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,CFID,CFPARAMS,SEASON


Jerry Sichting


,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,...,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,CFID,CFPARAMS,SEASON


In [172]:
master_dict

{29700009: {'home': {'team_id': 1610612745,
   'players': {'Mario Elie': {'sec_played': 1653, 'stats': {}},
    'Kevin Willis': {'sec_played': 1879, 'stats': {}},
    'Hakeem Olajuwon': {'sec_played': 2124, 'stats': {}},
    'Clyde Drexler': {'sec_played': 2022, 'stats': {}},
    'Matt Maloney': {'sec_played': 1449, 'stats': {}},
    'Rodrick Rhodes': {'sec_played': 1240, 'stats': {}},
    'Charles Barkley': {'sec_played': 1353, 'stats': {}},
    'Matt Bullard': {'sec_played': 404, 'stats': {}},
    'Brent Price': {'sec_played': 1431, 'stats': {}},
    'Eddie Johnson': {'sec_played': 845, 'stats': {}},
    'Emanual Davis': {'sec_played': 0, 'stats': {}},
    'Othella Harrington': {'sec_played': 0, 'stats': {}}}},
  'away': {'team_id': 1610612739,
   'players': {'Wesley Person': {'sec_played': 2132, 'stats': {}},
    'Danny Ferry': {'sec_played': 1884, 'stats': {}},
    'Zydrunas Ilgauskas': {'sec_played': 2092, 'stats': {}},
    'Derek Anderson': {'sec_played': 1189, 'stats': {}},
    

In [179]:
type(season_stats.loc[5,"SEASON"])

numpy.int64

In [182]:
int(str(games.loc[0, "SEASON_ID"])[1:]) == season_stats.loc[5,"SEASON"]

True

In [205]:
boxscores.loc[(boxscores["PLAYER_ID"] == 1279)]

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,SPD,...,CFGM,CFGA,CFG_PCT,UFGM,UFGA,UFG_PCT,FG_PCT,DFGM,DFGA,DFG_PCT
585,29700015,1610612748,MIA,Miami,1279,Jeff Bzdelik,NaN,,0:00,0.0,...,0,0,0.0,0,0,0.0,0.0,0,0,0.0
4536,29700185,1610612748,MIA,Miami,1279,Jeff Bzdelik,NaN,,0:00,0.0,...,0,0,0.0,0,0,0.0,0.0,0,0,0.0
8358,29700320,1610612748,MIA,Miami,1279,Jeff Bzdelik,NaN,,0:00,0.0,...,0,0,0.0,0,0,0.0,0.0,0,0,0.0
8834,29700330,1610612748,MIA,Miami,1279,Jeff Bzdelik,NaN,,0:00,0.0,...,0,0,0.0,0,0,0.0,0.0,0,0,0.0
9066,29700341,1610612748,MIA,Miami,1279,Jeff Bzdelik,NaN,,0:00,0.0,...,0,0,0.0,0,0,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234500,20201121,1610612743,DEN,Denver,1279,Jeff Bzdelik,NaN,,0:00,0.0,...,0,0,0.0,0,0,0.0,0.0,0,0,0.0
235451,20201149,1610612743,DEN,Denver,1279,Jeff Bzdelik,NaN,,0:00,0.0,...,0,0,0.0,0,0,0.0,0.0,0,0,0.0
235826,20201156,1610612743,DEN,Denver,1279,Jeff Bzdelik,NaN,,0:00,0.0,...,0,0,0.0,0,0,0.0,0.0,0,0,0.0
236543,20201176,1610612743,DEN,Denver,1279,Jeff Bzdelik,NaN,,0:00,0.0,...,0,0,0.0,0,0,0.0,0.0,0,0,0.0
